Earlier, we covered a simple method to implement a ground filter by classifying the minimum point in each cell as a ground point. This demonstrated some key concepts in LiDAR analysis, such as extracting summary data from grid cells. But we want something more robust.

Zhang et al (2003) implements a progressive morphological ground filter, one of many methods to filter ground points from unclassified LiDAR point clouds.

http://users.cis.fiu.edu/~chens/PDF/TGRS.pdf

In [21]:
import laspy
from scipy.interpolate import NearestNDInterpolator

In [3]:
%%capture
las = laspy.file.File("data/sample.las")

%run Canopy_Height_Model.ipynb

In [4]:
# INPUT
c = 1
b = 2
k = [0, 1, 2, 3, 4, 5, 6, 7, 8]
s = 0.08
dh0 = 0.25
dhmax = 2.5
max_win = 2 * max(k) + 1

Steps 1, 2 and 3 are already completed for us in the `grid` function, but we never dealt with cells that had no data.

In [1]:
gridded_las = grid(las, 0.2)

NameError: name 'grid' is not defined

In [80]:
group_df = gridded_las.groupby(['bins_x', 'bins_y'])
plot_mat = group_df.agg({'z': 'min'}).reset_index().pivot('bins_y', 'bins_x', 'z')


df = plot_mat.stack().rename_axis(['y', 'x']).reset_index(name='z')

coords = np.array(df[['x', 'y']])

z = np.array(df['z'])

z_interp = NearestNDInterpolator(coords, z)

In [84]:
a = np.where(np.isnan(np.array(plot_mat)))

z_interp(a[0], a[1])

array([ 490.93,  479.86,  479.6 , ...,  532.96,  526.94,  526.98])